In [1]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset <- fread("~/datasets/competencia_02_crudo.csv.gz" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-11-16 02:54:38 UTC"

Loading required package: data.table



,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,758070,40.5,1454677,77.7,1454677,77.7
Vcells,722123746,5509.4,1017364551,7761.9,845976803,6454.3


[1] "2025-11-16 02:56:07 UTC"

In [2]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


PRIMERO PREDIGO EL MEJOR CORTE CON LA VALIDACIÓN

In [3]:
# require( "data.table" )

setwd("/content/buckets/b1/exp")
# Carga de predicciones de probabilidad (202104)
p0 <- fread("./z_apo-12/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob0 = prob)]
p1 <- fread("./z_apo-01/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob1 = prob)]
p2 <- fread("./z_apo-03/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob2 = prob)]
p3 <- fread("./z_apo-05/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob3 = prob)]
p4 <- fread("./z_apo-07/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob4 = prob)]
p5 <- fread("./z_apo-08/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob5 = prob)]
p6 <- fread("./z_apo-09/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob6 = prob)]
p7 <- fread("./z_apo-10/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob7 = prob)]
p8 <- fread("./z_apo-11/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob8 = prob)]
p9 <- fread("./apo_exp005/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob9 = prob)]
p10 <- fread("./apo_exp006/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob10 = prob)]

# S/Tratamiento	IPC         	IPIM	UVA	         TC_BLUE	TC_OFICIAL	     TC Real Multilateral	ripte_indice	tasa interés real
# 411200000	   416000000	410000000	415600000	407200000	   409200000	     414800000	         415200000		409600000

# 1. Encontrar la ganancia máxima para cada modelo
g0_max <- 411200000
g1_max <- 415600000
g2_max <- 416000000
g3_max <- 410000000
g4_max <- 409200000
g5_max <- 407200000
g6_max <- 414800000
g7_max <- 415200000
g8_max <- 409600000
g9_max <- 409200000
g10_max <- 411600000

# Definir los modelos A ENSAMBLAR y sus ganancias
ganancias_a_ensamblar <- c(g0_max, g1_max, g2_max, g3_max, g4_max, g5_max, g6_max, g7_max, g8_max, g9_max, g10_max)

# Calcular la suma total de las ganancias de los modelos seleccionados
suma_total_ganancia <- sum(ganancias_a_ensamblar)

# 4. Calcular los pesos de ponderación
w0 <- g0_max / suma_total_ganancia
w1 <- g1_max / suma_total_ganancia
w2 <- g2_max / suma_total_ganancia
w3 <- g3_max / suma_total_ganancia
w4 <- g4_max / suma_total_ganancia
w5 <- g5_max / suma_total_ganancia
w6 <- g6_max / suma_total_ganancia
w7 <- g7_max / suma_total_ganancia
w8 <- g8_max / suma_total_ganancia
w9 <- g9_max / suma_total_ganancia
w10 <- g10_max / suma_total_ganancia

cat("--- Pesos de Ponderación (Basados en Ganancia Máxima Test 202106) ---\n")
cat(sprintf("Peso S/Tratamiento (w0): %.4f\n", w0))
cat(sprintf("Peso UVA (w1): %.4f\n", w1))
cat(sprintf("Peso IPC (w2): %.4f\n", w2))
cat(sprintf("Peso IPIM (w3): %.4f\n", w3))
cat(sprintf("Peso TC_OFICIAL (w4): %.4f\n", w4))
cat(sprintf("Peso TC_BLUE (w5): %.4f\n", w5))
cat(sprintf("Peso TC Real Multilateral (w6): %.4f\n", w6))
cat(sprintf("Peso ripte_indice (w7): %.4f\n", w7))
cat(sprintf("Peso tasa interés real (w8): %.4f\n", w8))
cat(sprintf("Peso meses impares (w9): %.4f\n", w9))
cat(sprintf("Peso meses pares (w10): %.4f\n", w10))

cat(sprintf("Suma de Pesos: %.4f\n", w0 + w1 + w2 + w3 + w4 + w5 + w6 + w7 + w8 + w9 + w10))
cat("----------------------------------------------------------------\n")


# Lista de Data.Tables a ensamblar
tablas_a_ensamblar <- list(p0, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10)

# Combinar las probabilidades
ensamble <- Reduce(function(x, y) merge(x, y, by=c("numero_de_cliente","foto_mes")),
                   tablas_a_ensamblar)

# Calcular la probabilidad ponderada (Weighted Ensemble)
ensamble[, prob_ponderada := (w0 * prob0) + (w1 * prob1) + (w2 * prob2) + (w3 * prob3) + (w4 * prob4) + (w5 * prob5) + (w6 * prob6) + (w7 * prob7) + (w8 * prob8) + (w9 * prob9) + (w10 * prob10)]

# Preparar el resultado final (ranking)
prediccion_202106_ensemble_ponderado <- ensamble[, .(numero_de_cliente, foto_mes, prob = prob_ponderada)]

cat("Predicción ensemble ponderada (202106) calculada con éxito.\n")

--- Pesos de Ponderación (Basados en Ganancia Máxima Test 202106) ---
Peso S/Tratamiento (w0): 0.0908
Peso UVA (w1): 0.0918
Peso IPC (w2): 0.0918
Peso IPIM (w3): 0.0905
Peso TC_OFICIAL (w4): 0.0903
Peso TC_BLUE (w5): 0.0899
Peso TC Real Multilateral (w6): 0.0916
Peso ripte_indice (w7): 0.0917
Peso tasa interés real (w8): 0.0904
Peso meses impares (w9): 0.0903
Peso meses pares (w10): 0.0909
Suma de Pesos: 1.0000
----------------------------------------------------------------
Predicción ensemble ponderada (202106) calculada con éxito.


In [4]:
realidad_evaluar <- function( tb_ranking ) {

  # Agrupar para calcular las cantidades (qty) solo de los envíos (Predicted = 1L)
  tbl <- tb_ranking[Predicted==1L, list("qty"=.N), list(clase_ternaria)]

  res <- list()
  res$total <- tbl[, sum(qty * ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]

  return( res )
}

In [5]:
require( "data.table" )


PARAM <- list(
    future = 202106,
    cortes = seq(6000, 13000, by= 100) # Tu rango de envíos
)

# Preparar el Data.table de Realidad (drealidad)
drealidad <- dataset[foto_mes == PARAM$future,
                     list(numero_de_cliente, foto_mes, clase_ternaria)]

# Crear la columna 'fold' para la división Público/Privado
# solo evalúo la ganancia total.
drealidad[, ultimo_digito := numero_de_cliente %% 10 ]
drealidad[, fold := 1L ]
drealidad[ultimo_digito >= 7, fold := 2L ]
drealidad[, ultimo_digito := NULL ] # Limpiar columna temporal

# Preparar el Data.table de Predicción (Ranking)
# Forzar tipo de dato a Integer para asegurar el merge
drealidad[, numero_de_cliente := as.integer(numero_de_cliente)]
prediccion_202106_ensemble_ponderado[, numero_de_cliente := as.integer(numero_de_cliente)]
tb_prediccion_ensemble <- prediccion_202106_ensemble_ponderado[, list(numero_de_cliente, prob)]

# 4. Unir el ranking de probabilidad (ensemble) con la realidad y el fold
drealidad_eval <- merge(drealidad,
                        tb_prediccion_ensemble,
                        by = "numero_de_cliente",
                        all.x = FALSE) # Usamos INNER JOIN para evitar clientes sin predicción

# Ordenar el ranking una sola vez
setorder(drealidad_eval, -prob)

# Inicializar la tabla para guardar la curva de ganancia
ganancias_ensemble_val <- data.table(
    envios = integer(),
    ganancia_total = numeric()
)


cat("\n--- Curva de Ganancia del Super-Ensemble (202104) ---\n")

for (envios in PARAM$cortes) {

    # Marcar la predicción binaria para el corte actual
    drealidad_eval[, Predicted := 0L] # Reseteamos
    drealidad_eval[1:envios, Predicted := 1L] # Marcamos el corte

    # Llamar a la función de evaluación
    res <- realidad_evaluar(drealidad_eval)

    # Guardar y luego imprimir el resultado del corte
    ganancias_ensemble_val <- rbind(ganancias_ensemble_val,
                                     list(envios, res$total))

    cat(sprintf("Corte %6d: Ganancia Total = %s\n",
                envios,
                format(res$total, big.mark = ".", decimal.mark = ",")))

    #  Limpiar la columna Predicted (Opcional, para el next loop)
    drealidad_eval[, Predicted := NULL]
}

corte_optimo <- ganancias_ensemble_val[which.max(ganancia_total)]

envios_optimos <- corte_optimo$envios
ganancia_maxima <- corte_optimo$ganancia_total

cat("\n------------------------------------------------------------\n")
cat("✅ CORTE ÓPTIMO EN VALIDACIÓN TEST (202106):\n")
cat(sprintf("   Envíos Óptimos: %d\n", envios_optimos))
cat(sprintf("   Ganancia Máxima: %s\n", format(ganancia_maxima, big.mark = ".", decimal.mark = ",")))
cat("------------------------------------------------------------\n")


--- Curva de Ganancia del Super-Ensemble (202104) ---
Corte   6000: Ganancia Total = 384.800.000
Corte   6100: Ganancia Total = 386.800.000
Corte   6200: Ganancia Total = 386.400.000
Corte   6300: Ganancia Total = 387.600.000
Corte   6400: Ganancia Total = 3,88e+08
Corte   6500: Ganancia Total = 388.400.000
Corte   6600: Ganancia Total = 393.600.000
Corte   6700: Ganancia Total = 393.200.000
Corte   6800: Ganancia Total = 392.800.000
Corte   6900: Ganancia Total = 392.400.000
Corte   7000: Ganancia Total = 393.600.000
Corte   7100: Ganancia Total = 395.600.000
Corte   7200: Ganancia Total = 395.200.000
Corte   7300: Ganancia Total = 395.600.000
Corte   7400: Ganancia Total = 397.600.000
Corte   7500: Ganancia Total = 400.400.000
Corte   7600: Ganancia Total = 4e+08
Corte   7700: Ganancia Total = 4,02e+08
Corte   7800: Ganancia Total = 403.200.000
Corte   7900: Ganancia Total = 403.600.000
Corte   8000: Ganancia Total = 404.800.000
Corte   8100: Ganancia Total = 405.200.000
Corte   820

In [6]:
ensamble

numero_de_cliente,foto_mes,prob0,prob1,prob2,prob3,prob4,prob5,prob6,prob7,prob8,prob9,prob10,prob_ponderada
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
249221323,202106,0.0027317716,0.0013840289,0.0015022111,0.002543428,0.0019846842,0.001782567,0.003432884,0.0020334960,0.0032670506,0.004197073,0.0018630309,0.002427354
249227600,202106,0.0009660755,0.0008749965,0.0012077847,0.001597583,0.0013267584,0.001327892,0.001263676,0.0015860152,0.0011643513,0.001084849,0.0010056061,0.001218519
249234235,202106,0.3245937640,0.4331994713,0.4939337097,0.463809935,0.4851851019,0.467617326,0.321165471,0.4271605198,0.2989649928,0.460771151,0.2267537423,0.400258878
249244449,202106,0.0108999933,0.0109479901,0.0161825028,0.010028989,0.0157672202,0.019735881,0.008719612,0.0152506203,0.0097281009,0.007432791,0.0057910780,0.011860419
249244739,202106,0.0113868865,0.0097401411,0.0139974524,0.018461904,0.0101223102,0.012119189,0.009544977,0.0127457873,0.0104910130,0.013475679,0.0080656603,0.011829321
249255456,202106,0.0087435657,0.0085368060,0.0066526063,0.005211585,0.0062138961,0.005099611,0.007308347,0.0077434939,0.0049676262,0.006722327,0.0103494901,0.007055532
249255747,202106,0.0062809345,0.0039202670,0.0028034771,0.004398955,0.0050393344,0.002565505,0.004817620,0.0031106223,0.0048490857,0.013347614,0.0162092430,0.006114934
249257429,202106,0.0310592591,0.0334583875,0.0381846559,0.039982275,0.0318570463,0.026589474,0.035030647,0.0345033007,0.0294843342,0.037128363,0.0405823442,0.034362294
249273979,202106,0.0051296468,0.0048976867,0.0057238782,0.005869210,0.0062076463,0.006362563,0.005142118,0.0075350796,0.0043593699,0.004415931,0.0038451291,0.005408882


In [7]:
tb_prediccion <- prediccion_202106_ensemble_ponderado[foto_mes == 202106, .(numero_de_cliente, foto_mes, prob)]
setorder(tb_prediccion, -prob)

In [8]:
# carpeta de salida en el espacio local de Colab
dir.create("ensambles", showWarnings=FALSE)

envios <- 11600  # (asegurate de tener esta variable definida)

# marco los primeros
tb_prediccion[, Predicted := 0L]
tb_prediccion[1:envios, Predicted := 1L]

# ruta de salida absoluta
archivo_kaggle <- paste0("./ensambles/KA_ensemble_ponderado", envios, ".csv")

# guardo el CSV
fwrite(
  tb_prediccion[, .(numero_de_cliente, Predicted)],
  file = archivo_kaggle,
  sep = ","
)

cat("Archivo guardado en:", archivo_kaggle, "\n")

# si querés ver que realmente existe:
list.files("./ensambles")

Archivo guardado en: ./ensambles/KA_ensemble_ponderado11600.csv 


[1] "KA_ensemble_ponderado11500.csv" "KA_ensemble_ponderado11600.csv"

## PREDICCIÓN 202108

In [38]:
require( "data.table" )

setwd("/content/buckets/b1/exp")
# Carga de predicciones de probabilidad (202108)
f1 <- fread("./c_apo_01/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob1 = prob)]
f2 <- fread("./c_apo_02/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob2 = prob)]
f3 <- fread("./c_apo_03/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob3 = prob)]
f4 <- fread("./c_apo_04/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob4 = prob)]
f5 <- fread("./c_apo_05/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob5 = prob)]
f6 <- fread("./c_apo_06/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob6 = prob)]
f7 <- fread("./c_apo_07/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob7 = prob)]
f8 <- fread("./c_apo_08/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob8 = prob)]
f9 <- fread("./c_apo_09/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob9 = prob)]
f10 <- fread("./c_apo_10/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob10 = prob)]
f11 <- fread("./c_apo_11/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob11 = prob)]
f12 <- fread("./c_apo_12/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob12 = prob)]
f13 <- fread("./c_apo_13/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob13 = prob)]
f14 <- fread("./c_apo_14/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob14 = prob)]
f15 <- fread("./c_apo_15/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob15 = prob)]
f16 <- fread("./c_apo_16/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob16 = prob)]
f17 <- fread("./c_apo_17/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob17 = prob)]
f18 <- fread("./c_apo_18/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob18 = prob)]
f19 <- fread("./c_apo_19/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob19 = prob)]
f20 <- fread("./c_apo_20/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob20 = prob)]
f21 <- fread("./exp005/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob21 = prob)]
f22 <- fread("./exp006/prediccion.txt")[, .(numero_de_cliente, foto_mes, prob22 = prob)]

# S/Tratamiento	IPC         	IPIM	UVA	         TC_BLUE	TC_OFICIAL	     TC Real Multilateral	ripte_indice	tasa interés real
# 411200000	   416000000	410000000	415600000	407200000	   409200000	     414800000	         415200000		409600000

# 1. Encontrar la ganancia máxima para cada modelo
g_ipc <- 416000000
g_uva <- 415600000
g_tcrm <- 414800000
g_ripte <- 415200000
g_pares <- 411600000
g_impares <- 409200000

# Definir los modelos A ENSAMBLAR y sus ganancias
ganancias_a_ensamblar <- c(g_ipc + g_uva + g_tcrm + g_ripte + g_pares + g_impares)

# Calcular la suma total de las ganancias de los modelos seleccionados
suma_total_ganancia <- sum(ganancias_a_ensamblar)

# 4. Calcular los pesos de ponderación
w1 <- (g_ipc/5) / suma_total_ganancia
w2 <- (g_uva/5) / suma_total_ganancia
w3 <- (g_tcrm/5) / suma_total_ganancia
w4 <- (g_ripte/5) / suma_total_ganancia
w5 <- (g_ipc/5) / suma_total_ganancia
w6 <- (g_uva/5) / suma_total_ganancia
w7 <- (g_tcrm/5) / suma_total_ganancia
w8 <- (g_ripte/5) / suma_total_ganancia
w9 <- (g_ipc/5) / suma_total_ganancia
w10 <- (g_uva/5) / suma_total_ganancia
w11 <- (g_tcrm/5) / suma_total_ganancia
w12 <- (g_ripte/5) / suma_total_ganancia
w13 <- (g_ipc/5) / suma_total_ganancia
w14 <- (g_uva/5) / suma_total_ganancia
w15 <- (g_tcrm/5) / suma_total_ganancia
w16 <- (g_ripte/5) / suma_total_ganancia
w17 <- (g_ipc/5) / suma_total_ganancia
w18 <- (g_uva/5) / suma_total_ganancia
w19 <- (g_tcrm/5) / suma_total_ganancia
w20 <- (g_ripte/5) / suma_total_ganancia
w21 <- g_impares / suma_total_ganancia
w22 <- g_pares / suma_total_ganancia

cat("--- Pesos de Ponderación (Basados en Ganancia Máxima Test 202106) ---\n")
cat(sprintf("Peso IPC seed 100003 (w1): %.4f\n", w1))
cat(sprintf("Peso UVA seed 100003 (w2): %.4f\n", w2))
cat(sprintf("Peso TCRM seed 100003 (w3): %.4f\n", w3))
cat(sprintf("Peso RIPTE seed 100003 (w4): %.4f\n", w4))
cat(sprintf("Peso IPC seed 111109 (w5): %.4f\n", w5))
cat(sprintf("Peso UVA seed 111109 (w6): %.4f\n", w6))
cat(sprintf("Peso TCRM seed 111109 (w7): %.4f\n", w7))
cat(sprintf("Peso RIPTE seed 111109 (w8): %.4f\n", w8))
cat(sprintf("Peso IPC seed 444449 (w9): %.4f\n", w9))
cat(sprintf("Peso UVA seed 444449 (w10): %.4f\n", w10))
cat(sprintf("Peso TCRM seed 444449 (w11): %.4f\n", w11))
cat(sprintf("Peso RIPTE seed 444449 (w12): %.4f\n", w12))
cat(sprintf("Peso IPC seed 555557 (w13): %.4f\n", w13))
cat(sprintf("Peso UVA seed 555557 (w14): %.4f\n", w14))
cat(sprintf("Peso TCRM seed 555557 (w15): %.4f\n", w15))
cat(sprintf("Peso RIPTE seed 555557 (w16): %.4f\n", w16))
cat(sprintf("Peso IPC seed 999961 (w17): %.4f\n", w17))
cat(sprintf("Peso UVA seed 999961 (w18): %.4f\n", w18))
cat(sprintf("Peso TCRM seed 999961 (w19): %.4f\n", w19))
cat(sprintf("Peso RIPTE seed 999961 (w20): %.4f\n", w20))
cat(sprintf("Peso meses impares seed 100003 (w21): %.4f\n", w21))
cat(sprintf("Peso meses pares seed 100003 (w22): %.4f\n", w22))

cat(sprintf("Suma de Pesos: %.4f\n", w1 + w2 + w3 + w4 + w5 + w6 + w7 + w8 + w9 + w10 + w11 + w12 + w13 + w14 + w15 + w16 + w17 + w18 + w19 + w20 + w21 + w22))
cat("----------------------------------------------------------------\n")


# Lista de Data.Tables a ensamblar
tablas_a_ensamblar <- list(f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f14, f15, f16, f17, f18, f19, f20, f21, f22)

# Combinar las probabilidades
ensamble <- Reduce(function(x, y) merge(x, y, by=c("numero_de_cliente","foto_mes")),
                   tablas_a_ensamblar)

# Calcular la probabilidad ponderada (Weighted Ensemble)
ensamble[, prob_ponderada := (w1 * prob1) + (w2 * prob2) + (w3 * prob3) + (w4 * prob4) + (w5 * prob5) + (w6 * prob6) + (w7 * prob7) + (w8 * prob8) + (w9 * prob9) + (w10 * prob10) + (w11 * prob11) + (w12 * prob12) + (w13 * prob13) + (w14 * prob14) + (w15 * prob15) + (w16 * prob16) + (w17 * prob17) + (w18 * prob18) + + (w19 * prob19) + (w20 * prob20) + (w21 * prob21) + (w22 * prob22)]

# Preparar el resultado final (ranking)
prediccion_202108_ensemble_ponderado <- ensamble[, .(numero_de_cliente, foto_mes, prob = prob_ponderada)]

cat("Predicción ensemble ponderada (202108) calculada con éxito.\n")

--- Pesos de Ponderación (Basados en Ganancia Máxima Test 202106) ---
Peso IPC seed 100003 (w1): 0.0335
Peso UVA seed 100003 (w2): 0.0335
Peso TCRM seed 100003 (w3): 0.0334
Peso RIPTE seed 100003 (w4): 0.0335
Peso IPC seed 111109 (w5): 0.0335
Peso UVA seed 111109 (w6): 0.0335
Peso TCRM seed 111109 (w7): 0.0334
Peso RIPTE seed 111109 (w8): 0.0335
Peso IPC seed 444449 (w9): 0.0335
Peso UVA seed 444449 (w10): 0.0335
Peso TCRM seed 444449 (w11): 0.0334
Peso RIPTE seed 444449 (w12): 0.0335
Peso IPC seed 555557 (w13): 0.0335
Peso UVA seed 555557 (w14): 0.0335
Peso TCRM seed 555557 (w15): 0.0334
Peso RIPTE seed 555557 (w16): 0.0335
Peso IPC seed 999961 (w17): 0.0335
Peso UVA seed 999961 (w18): 0.0335
Peso TCRM seed 999961 (w19): 0.0334
Peso RIPTE seed 999961 (w20): 0.0335
Peso meses impares seed 100003 (w21): 0.1648
Peso meses pares seed 100003 (w22): 0.1658
Suma de Pesos: 1.0000
----------------------------------------------------------------
Predicción ensemble ponderada (202108) calculada 

In [39]:
tb_prediccion <- prediccion_202108_ensemble_ponderado[foto_mes == 202108, .(numero_de_cliente, foto_mes, prob)]
setorder(tb_prediccion, -prob)

In [40]:
tb_prediccion

numero_de_cliente,foto_mes,prob
<int>,<int>,<dbl>
709048487,202108,0.9957962
1323711255,202108,0.9952720
1248124471,202108,0.9943922
595282122,202108,0.9941544
920436482,202108,0.9937606
873972753,202108,0.9912070
1332810077,202108,0.9903854
254111380,202108,0.9897509
847823805,202108,0.9896930


In [36]:
# carpeta de salida en el espacio local de Colab
dir.create("ensambles", showWarnings=FALSE)

envios <- 11500 

# marco los primeros
tb_prediccion[, Predicted := 0L]
tb_prediccion[1:envios, Predicted := 1L]

archivo_kaggle <- paste0("./ensambles/eze_submit_03.csv")

# 2. Guardar solo los 'numero_de_cliente' donde Predicted == 1
#    y SIN encabezado (col.names = FALSE)

fwrite(
  # Filtrar SÓLO las filas que tienen Predicted == 1
  # y seleccionar SÓLO la columna numero_de_cliente
  tb_prediccion[Predicted == 1, .(numero_de_cliente)], 
  file = archivo_kaggle,
  sep = ",",
  col.names = FALSE  # <-- Esto elimina el encabezado
)

cat("Archivo guardado en:", archivo_kaggle, "\n")
list.files("./ensambles")

Archivo guardado en: ./ensambles/eze_submit_02.csv 


[1] "eze_submit_01.csv"              "eze_submit_02.csv"             
[3] "KA_ensemble_ponderado11500.csv" "KA_ensemble_ponderado11600.csv"

In [37]:
require( "data.table" )

# La ruta completa al archivo (asumiendo que estás en /content/buckets/b1/exp)
archivo_kaggle <- "./ensambles/eze_submit_03.csv"

# Leer el archivo con fread
# Usamos header=FALSE para indicar que no hay nombres de columna
clientes_predichos <- fread(
  file = archivo_kaggle, 
  header = FALSE, 
  col.names = "numero_de_cliente" # Le asignamos un nombre para que sea más claro
)

# Mostrar las primeras filas de los clientes predichos
cat("--- Primeros 10 Clientes Predichos ---\n")
print(head(clientes_predichos, 10))

# Verificar cuántos clientes se cargaron (debería ser 11500)
cat("\nNúmero total de envíos predichos cargados:", nrow(clientes_predichos), "\n")

--- Primeros 10 Clientes Predichos ---
    numero_de_cliente
                <int>
 1:         709048487
 2:        1323711255
 3:        1248124471
 4:         595282122
 5:         920436482
 6:         873972753
 7:        1332810077
 8:         254111380
 9:         847823805
10:        1343877265

Número total de envíos predichos cargados: 11500 
